In [1]:
import numpy as np

In [5]:
file = '/Volumes/T7/Voxel_Bank/Data/NifTi/V001/vol_2.0res_3.0sl/02_Rejected/stats.npy'
data = np.load(file, allow_pickle=True)
print(data)

[[100.         100.         100.         100.         100.
   85.71428571 100.          85.71428571 100.         100.
  100.         100.         100.        ]]


In [32]:
import os
import numpy as np
from collections import defaultdict

volunteer_all = ['V001', 'V002', 'V003', 'V004', 'V005', 'V006', 'V007', 'V008', 'V009','V010','V011']
resolution_all = ['vol_2.0res_3.0sl', 'vol_2.0res_5.0sl', 'vol_2.0res_8.0sl', 
                  'vol_2.5res_3.0sl', 'vol_2.5res_5.0sl', 'vol_2.5res_8.0sl', 
                  'vol_3.0res_3.5sl', 'vol_3.0res_5.5sl', 'vol_3.0res_8.0sl']

base_path = '/Volumes/T7/Voxel_Bank/Data/NifTi'

# To store total accepted images and counts per resolution
resolution_stats = defaultdict(list)

for resolution in resolution_all:
    for volunteer in volunteer_all:
        file_path = os.path.join(base_path, volunteer, resolution, '02_Rejected', 'stats.npy')
        if os.path.isfile(file_path):
            data = np.load(file_path, allow_pickle=True)
            
            # Calculate number of accepted images: sum over directions
            accepted_images = np.sum(data * 7 / 100)
            resolution_stats[resolution].append(accepted_images)
            
        else:
            print(f"File not found: {file_path}")
        precent = accepted_images / 91 * 100 if accepted_images > 0 else 0
        print(f"Processed {volunteer} for {resolution}: {accepted_images:.1f} accepted images, {precent}% of total")

# Report total and average per resolution
print("\n=== Acceptance Summary per Resolution ===")
for resolution, accepted_list in resolution_stats.items():
    total_images = sum(accepted_list)
    num_subjects = len(accepted_list)
    average_images = total_images / num_subjects 
    print(f"{resolution:20s} | Subjects: {num_subjects:2d} | Total accepted: {total_images:6.1f} | Avg per subj: {average_images:.1f}")
    print(f"Percentage of total: {total_images / (91 * 11) * 100:.2f}%\n")
    print(f"Percent of rejected images: {(1001 - total_images) / 1001 * 100:.2f}\n")

Processed V001 for vol_2.0res_3.0sl: 89.0 accepted images, 97.8021978021978% of total
Processed V002 for vol_2.0res_3.0sl: 86.0 accepted images, 94.5054945054945% of total
Processed V003 for vol_2.0res_3.0sl: 87.0 accepted images, 95.6043956043956% of total
Processed V004 for vol_2.0res_3.0sl: 81.0 accepted images, 89.01098901098901% of total
Processed V005 for vol_2.0res_3.0sl: 90.0 accepted images, 98.9010989010989% of total
Processed V006 for vol_2.0res_3.0sl: 90.0 accepted images, 98.9010989010989% of total
Processed V007 for vol_2.0res_3.0sl: 88.0 accepted images, 96.7032967032967% of total
Processed V008 for vol_2.0res_3.0sl: 86.0 accepted images, 94.5054945054945% of total
Processed V009 for vol_2.0res_3.0sl: 90.0 accepted images, 98.9010989010989% of total
Processed V010 for vol_2.0res_3.0sl: 91.0 accepted images, 100.0% of total
Processed V011 for vol_2.0res_3.0sl: 76.0 accepted images, 83.51648351648352% of total
Processed V001 for vol_2.0res_5.0sl: 90.0 accepted images, 98.9

In [4]:
import os
import pydicom
from datetime import datetime
import pandas as pd

# Root path
root_path = '/Volumes/T7/Voxel_Bank/Data/Dicoms/'

# Prepare results list
results = []

# Loop over volunteer folders
for volunteer in sorted(os.listdir(root_path)):
    volunteer_path = os.path.join(root_path, volunteer)
    if not os.path.isdir(volunteer_path) or not volunteer.startswith('V'):
        continue  # skip non-volunteer folders

    # Loop over resolution folders like 'vol_2.0res_3.0sl'
    for res_folder in sorted(os.listdir(volunteer_path)):
        res_path = os.path.join(volunteer_path, res_folder)
        if not os.path.isdir(res_path) or not res_folder.startswith('vol_'):
            continue

        # Find the first subfolder that contains .IMA files
        dcm_subfolder = None
        for subfolder in os.listdir(res_path):
            subfolder_path = os.path.join(res_path, subfolder)
            if os.path.isdir(subfolder_path):
                files = os.listdir(subfolder_path)
                ima_files = [f for f in files if f.lower().endswith('.ima') or not '.' in f]
                if ima_files:
                    dcm_subfolder = subfolder_path
                    break  # use first one found

        if dcm_subfolder is None:
            print(f"No DICOM folder found for {volunteer}/{res_folder}")
            continue

        # Parse .IMA files and get acquisition times
        acq_times = []
        for file in os.listdir(dcm_subfolder):
            if not file.lower().endswith('.ima') and '.' in file:
                continue
            try:
                ds = pydicom.dcmread(os.path.join(dcm_subfolder, file), stop_before_pixels=True)
                acq_time = ds.get('AcquisitionTime')
                acq_date = ds.get('AcquisitionDate') or ds.get('StudyDate')
                if acq_time and acq_date:
                    dt_str = acq_date + acq_time.split('.')[0]
                    dt = datetime.strptime(dt_str, '%Y%m%d%H%M%S')
                    acq_times.append(dt)
            except Exception as e:
                continue  # skip unreadable files

        if len(acq_times) < 2:
            print(f"Not enough acquisition times for {volunteer}/{res_folder}")
            continue

        acq_times.sort()
        time_diffs = [(acq_times[i+1] - acq_times[i]).total_seconds() for i in range(len(acq_times)-1)]
        avg_time_between = sum(time_diffs) / len(time_diffs)
        total_scan_time = (acq_times[-1] - acq_times[0]).total_seconds()

        results.append({
            'Volunteer': volunteer,
            'ResolutionFolder': res_folder,
            'DICOM_Folder': os.path.basename(dcm_subfolder),
            'NumSlices': len(acq_times),
            'AvgTimeBetweenScans_sec': round(avg_time_between, 2),
            'TotalScanTime_sec': round(total_scan_time, 2),
        })

# Save summary to CSV
df = pd.DataFrame(results)
csv_path = os.path.join(root_path, 'scan_timing_summary.csv')
df.to_csv(csv_path, index=False)

print(f"✔️ Summary saved to {csv_path}")

✔️ Summary saved to /Volumes/T7/Voxel_Bank/Data/Dicoms/scan_timing_summary.csv


In [7]:
# report overall average and standard deviation Total SCan time 
total_scan_times = [res['TotalScanTime_sec'] for res in results]
overall_avg_scan_time = sum(total_scan_times) / len(total_scan_times) if total_scan_times else 0
overall_std_scan_time = np.std(total_scan_times) if total_scan_times else 0
print(f"Overall average scan time: {overall_avg_scan_time:.2f} seconds")
print(f"Overall standard deviation of scan times: {overall_std_scan_time:.2f} seconds")

# convert seconds to minutes for better readability
overall_avg_scan_time_min = overall_avg_scan_time / 60
overall_std_scan_time_min = overall_std_scan_time / 60
print(f"Overall average scan time: {overall_avg_scan_time_min:.2f} minutes")
print(f"Overall standard deviation of scan times: {overall_std_scan_time_min:.2f} minutes")

# average time between scans 
avg_time_between_scans = [res['AvgTimeBetweenScans_sec'] for res in results]
overall_avg_time_between = sum(avg_time_between_scans) / len(avg_time_between_scans) if avg_time_between_scans else 0
overall_std_time_between = np.std(avg_time_between_scans) if avg_time_between_scans else 0
print(f"Overall average time between scans: {overall_avg_time_between:.2f} seconds")
print(f"Overall standard deviation of time between scans: {overall_std_time_between:.2f} seconds")  



Overall average scan time: 357.12 seconds
Overall standard deviation of scan times: 60.50 seconds
Overall average scan time: 5.95 minutes
Overall standard deviation of scan times: 1.01 minutes
Overall average time between scans: 3.89 seconds
Overall standard deviation of time between scans: 0.77 seconds


In [22]:
data

array([[ 85.71428571, 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        , 100.        ,
         85.71428571, 100.        , 100.        , 100.        ,
        100.        ]])

In [24]:
89/91

0.978021978021978